### 实验0.5b-微调后模型，完成基础的多轮对话

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 确保CUDA可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 指定模型路径
model_path = "D:\上海交大本科阶段\大三上\自然语言处理\project\lora"
# 加载微调后模型 (SFT) 和 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map=None, 
    torch_dtype=torch.bfloat16
).to(device)


<>:7: SyntaxWarning: invalid escape sequence '\p'
<>:7: SyntaxWarning: invalid escape sequence '\p'
C:\Windows\Temp\ipykernel_12388\210006252.py:7: SyntaxWarning: invalid escape sequence '\p'
  model_path = "D:\上海交大本科阶段\大三上\自然语言处理\project\lora"
d:\ana\envs\machinelearn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.61it/s]


### 多轮对话    

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import TextIteratorStreamer
from threading import Thread
# 创建 TextStreamer 实例
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
# 临时存储流式生成的文本
# 全局对话历史
conversation_history = []

# 模型支持的最大 token 长度
MAX_MODEL_LENGTH = model.config.max_position_embeddings
print("MAX History length:", MAX_MODEL_LENGTH)
# 聊天机器人主函数
def chatbot():
    global conversation_history  # 声明使用全局变量
    print("欢迎使用 Qwen 聊天机器人！输入内容即可开始对话。\n输入 \\quit 结束会话，输入 \\newsession 清空对话历史并重新开始。\n")

    while True:
        # 用户输入
        user_input = input("用户: ").strip()
        
        print("用户:", user_input)
        # 退出会话
        if user_input == r"\quit":
            print("聊天机器人已退出，会话结束。")
            break
        
        # 开启新会话
        elif user_input == r"\newsession":
            conversation_history = []
            print("已清空对话历史，开启新的对话会话！")
            continue
        
        # 普通对话
        else:
            # 添加用户输入到对话历史
            conversation_history.append({"role": "user", "content": user_input})
            trim_conversation_history()  # 裁剪对话历史
            # 构造模型输入（整合你提供的逻辑）
            text = tokenizer.apply_chat_template(
                format_messages(conversation_history),
                tokenize=False,  # 不立即分词
                add_generation_prompt=True  # 添加生成提示
            )

            # 模型推理
            inputs = tokenizer(text, return_tensors="pt", padding=True).to(model.device)
            
            
            generation_kwargs = dict(inputs, max_new_tokens=400,  # 限制生成长度
                # max_length=1024,  # 限制最大长度
                do_sample= True,
                temperature=0.2,  # 调整生成的随机性
                top_k=5,  # 限制高概率单词的候选范围
                top_p=0.95,  # 核采样
                repetition_penalty=1.1,  # 惩罚重复生成
                eos_token_id=tokenizer.eos_token_id,  # 设置结束标记
                pad_token_id=tokenizer.eos_token_id,  # 设置填充标记
                streamer=streamer  # 设置 streamer 用于流式输出)
            )
            
            thread = Thread(target=model.generate, kwargs=generation_kwargs)
            thread.start()
            generated_text = ""
            print ("助手: ", end="", flush=True)
            for new_text in streamer:
                generated_text += new_text
                print(new_text, end="", flush=True)
            
            # 解码生成的响应
            # response = tokenizer.decode(outputs[0, inputs["input_ids"].size(1):], skip_special_tokens=True).strip()
            # print("助手回复:", response)
            
            # 添加助手响应到对话历史
            conversation_history.append({"role": "assistant", "content": generated_text})
            trim_conversation_history()
            # print("\n对话历史:", conversation_history)
            # 计算对话历史长度
            formatted_history = tokenizer.apply_chat_template(
                format_messages(conversation_history),
                tokenize=False,  # 返回未分词的字符串
                add_generation_prompt=False
            )
            
            # 对格式化后的字符串进行分词，获取 input_ids
            tokenized_history = tokenizer(formatted_history, return_tensors="pt", padding=True)
            input_length = tokenized_history["input_ids"].shape[1]
            print("\n对话历史长度:", input_length)
            
            print("\n --------------------")
            # 如果对话历史超出模型最大长度，则裁剪对话历史
            


# 格式化对话历史为 Qwen 支持的模板
def format_messages(history):
    """
    将对话历史格式化为模型支持的模板格式
    """
    formatted_messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are helpful and intelligent."}
    ]  # 初始包含系统指令
    formatted_messages.extend(history)  # 添加用户和助手的对话
    return formatted_messages

# 裁剪对话历史，保证长度不超过模型支持的最大长度
def trim_conversation_history():
    """
    当对话历史长度超过模型支持的最大长度时，裁剪最早的对话
    """
    global conversation_history
    
    # 使用 apply_chat_template 格式化对话历史
    formatted_history = tokenizer.apply_chat_template(
        format_messages(conversation_history),
        tokenize=False,  # 返回未分词的字符串
        add_generation_prompt=False
    )
    
    # 对格式化后的字符串进行分词，获取 input_ids
    tokenized_history = tokenizer(formatted_history, return_tensors="pt", padding=True)
    input_length = tokenized_history["input_ids"].shape[1]

    # 如果长度未超出最大值，不做裁剪
    if input_length <= MAX_MODEL_LENGTH:
        return

    # 超出长度时，逐步移除最早的对话，直到满足长度限制
    while input_length > MAX_MODEL_LENGTH:
        if conversation_history:  # 确保历史不为空
            conversation_history.pop(0)  # 移除最早的对话
        # 更新格式化后的对话历史并重新计算长度
        formatted_history = tokenizer.apply_chat_template(
            format_messages(conversation_history),
            tokenize=False,
            add_generation_prompt=False
        )
        tokenized_history = tokenizer(formatted_history, return_tensors="pt", padding=True)
        input_length = tokenized_history["input_ids"].shape[1]
        # print("裁剪对话历史，当前长度:", input_length)
# 启动聊天机器人
if __name__ == "__main__":
    chatbot()


d:\ana\envs\machinelearn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'tokenizer' is not defined